# Predicting Click-Through Rate Prediction 
## Random Forest

Ryan Koch, Sam Kahr, Julia Kang - AMLI 2019

## Import Libraries and Data

In [18]:
import pandas as pd
import numpy as np
import multiprocessing as mp
import psutil
import random
import datetime as datetime
import matplotlib.pyplot as plt

In [25]:
#read in the Avazu - criteo labs - csv file

df = pd.read_csv('/home/data/rand_sample_csv.csv')



# Data Exploration

In [27]:
df.head(5)

,Unnamed: 0,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,0,10004510652136496837,0,14102100,1005,0,543a539e,c7ca3108,3e814130,ecad2386,...,1,0,20352,320,50,2333,0,39,-1,157
1,1,10007164336863914220,1,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,...,1,0,15706,320,50,1722,0,35,-1,79
2,2,10076859283156800622,0,14102100,1002,0,f17ebd97,c4e18dd6,50e219e0,ecad2386,...,0,0,21692,216,36,2497,3,43,100151,42
3,3,10078825124049580646,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,...,1,0,15705,320,50,1722,0,35,-1,79
4,4,10085233430943183912,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,...,1,0,15701,320,50,1722,0,35,-1,79


In [8]:
df.shape

(404410, 25)

In [ ]:
# At which time do people click ads most frequently?
# where are most clicks coming from? 
# I wonder if there are any unique correlations from any or all combinations of each column to click?
# what correlations between seemingly unrelated columns (banner_pos and device type, etc.) exist?

In [9]:
df.describe()

,Unnamed: 0,id,click,hour,C1,banner_pos,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
count,404410.000000,4.044100e+05,404410.000000,4.044100e+05,404410.000000,404410.000000,404410.000000,404410.000000,404410.000000,404410.000000,404410.000000,404410.000000,404410.000000,404410.000000,404410.000000,404410.000000
mean,202204.500000,9.213896e+18,0.169991,1.410256e+07,1004.967372,0.288638,1.014695,0.331688,18844.936193,318.795856,60.076581,2112.733881,1.431426,227.891140,53239.462209,83.304955
std,116743.255519,5.319411e+18,0.375625,2.968127e+02,1.090207,0.504033,0.523959,0.855877,4947.526554,20.667022,47.023691,607.929983,1.325359,351.686105,49955.391770,70.251537
min,0.000000,7.306813e+13,0.000000,1.410210e+07,1001.000000,0.000000,0.000000,0.000000,375.000000,120.000000,20.000000,112.000000,0.000000,33.000000,-1.000000,1.000000
25%,101102.250000,4.607240e+18,0.000000,1.410230e+07,1005.000000,0.000000,1.000000,0.000000,16920.000000,320.000000,50.000000,1863.000000,0.000000,35.000000,-1.000000,23.000000
50%,202204.500000,9.218443e+18,0.000000,1.410260e+07,1005.000000,0.000000,1.000000,0.000000,20346.000000,320.000000,50.000000,2323.000000,2.000000,39.000000,100048.000000,61.000000
75%,303306.750000,1.382200e+19,0.000000,1.410281e+07,1005.000000,1.000000,1.000000,0.000000,21893.000000,320.000000,50.000000,2526.000000,3.000000,171.000000,100086.000000,101.000000
max,404409.000000,1.844673e+19,1.000000,1.410302e+07,1012.000000,7.000000,5.000000,5.000000,24043.000000,1024.000000,1024.000000,2757.000000,3.000000,1839.000000,100248.000000,255.000000


In [10]:
# looks like clicks were normalized to 1. I wonder what the stratification of the value 1 means?
# how are banner_pos, device_type, device_conn_type encoded?

In [29]:
# lets look at an individual user

df[df.id == 10007164336863914220]

,Unnamed: 0,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
1,1,10007164336863914220,1,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,...,1,0,15706,320,50,1722,0,35,-1,79


In [30]:
# lets look at an individual user

df[df.id == 10007164336863914220]

,Unnamed: 0,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
1,1,10007164336863914220,1,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,...,1,0,15706,320,50,1722,0,35,-1,79


In [31]:
df.dtypes

Unnamed: 0           int64
id                  uint64
click                int64
hour                 int64
C1                   int64
banner_pos           int64
site_id             object
site_domain         object
site_category       object
app_id              object
app_domain          object
app_category        object
device_id           object
device_ip           object
device_model        object
device_type          int64
device_conn_type     int64
C14                  int64
C15                  int64
C16                  int64
C17                  int64
C18                  int64
C19                  int64
C20                  int64
C21                  int64
dtype: object

In [32]:
# bummer the juicy features are objs probably strings, lets take a closer look

In [ ]:
# what do the columns mean?

id: ad identifier
click: 0/1 for non-click/click
hour: format is YYMMDDHH
C1 — anonymized categorical variable
banner_pos
site_id
site_domain
site_category
app_id
app_domain
app_category
device_id
device_ip
device_model
device_type
device_conn_type
C14-C21 — anonymized categorical variables

In [44]:

for col in df.columns.values:
    print( "{}".format(len(df[col].unique()))

404410
404410
2
240
7
7
2195
2172
21
2305
153
26
64913
262453
4369
5
4
2067
8
9
415
4
65
159
60
